<a href="https://colab.research.google.com/github/pcsilcan/pcd/blob/master/20202/pcd_20202_0402_crit_sections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install spin

# Critical Section
Queremos verificar las condiciones de correctitud de programas con sección crítica:
* Exclusión mutua
* No deadlock
* No livelock
* No starvation

In [4]:
%%writefile crit1.pml
byte turn = 1

active proctype P() {
    byte i = 0
    for (i : 1..20) {
        printf("NCR 1\n"); printf("NCR 2\n"); printf("NCR 3\n");

        turn == 1-> // await (expr -> op1 [: op2] === expr ? op1 : op2)
        printf("Critical 1\n"); printf("Critical 2\n"); printf("Critical 3\n")

        turn = 2
    }
}
active proctype Q() {
    byte i = 0
    for (i : 1..20) {
        printf("NCR 1\n"); printf("NCR 2\n"); printf("NCR 3\n");

        turn == 2-> // await

        printf("Critical 1\n"); printf("Critical 2\n"); printf("Critical 3\n")

        turn = 1
    }
}

Overwriting crit1.pml


In [ ]:
!spin crit1.pml

In [6]:
%%writefile crit1b.pml
byte turn = 1
byte cont = 0

active proctype P() {
    do
    :: turn == 1-> 
        cont++
        assert(cont < 2)
        cont--
        turn = 2
    od
}
active proctype Q() {
    do
    :: turn == 2->
        cont++
        assert(cont < 2)
        cont--
        turn = 1
    od
}

Writing crit1b.pml


In [7]:
!spin -run crit1b.pml


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 20 byte, depth reached 9, errors: 0
       10 states, stored
        1 states, matched
       11 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.000	equivalent memory usage for states (stored*(State-vector + overhead))
    0.291	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype P
	crit1b.pml:12, state 9, "-end-"
	(1 of 9 states)
unreached in proctype Q
	crit1b.pml:21, state 9, "-end-"
	(1 of 9 states)

pan: elapsed time 0 seconds


In [9]:
%%writefile crit2b.pml
bool wantp = false, wantq = false
byte cont = 0

active proctype P() {
    do
    :: !wantq -> 
        wantp = true
        cont++
        assert(cont < 2)
        cont--
        wantp = false
    od
}
active proctype Q() {
    do
    :: !wantp -> 
        wantq = true
        cont++
        assert(cont < 2)
        cont--
        wantq = false
    od
}

Writing crit2b.pml


In [10]:
!spin -run crit2b.pml

pan:1: assertion violated (cont<2) (at depth 7)
pan: wrote crit2b.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 20 byte, depth reached 11, errors: 1
       22 states, stored
       10 states, matched
       32 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.001	equivalent memory usage for states (stored*(State-vector + overhead))
    0.291	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0.01 seconds


In [14]:
%%writefile crit3b.pml
bool wantp = false, wantq = false

active proctype P() {
    do
    :: 
        wantp = true
        !wantq ->
        wantp = false
    od
}
active proctype Q() {
    do
    ::
        wantq = true
        !wantp ->
        wantq = false
    od
}

Overwriting crit3b.pml


In [12]:
!spin -run crit3b.pml

pan:1: invalid end state (at depth 4)
pan: wrote crit3b.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 20 byte, depth reached 5, errors: 1
        6 states, stored
        1 states, matched
        7 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.000	equivalent memory usage for states (stored*(State-vector + overhead))
    0.292	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds


In [13]:
!spin -t crit3b.pml

spin: trail ends after 5 steps
#processes: 2
		wantp = 1
		wantq = 1
  5:	proc  1 (Q:1) crit3b.pml:19 (state 2)
  5:	proc  0 (P:1) crit3b.pml:8 (state 2)
2 processes created
